In [151]:
import pandas as pd
import numpy
from sklearn.ensemble import RandomForestClassifier as rfc

#read in dataset
df = pd.read_csv("data.csv")
df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)


#values formatted
dfTrn, dfDev, dfTes = numpy.split(df, [int(.15*len(df)), int(.7*len(df))])
DTrn, DDev, DTes = numpy.split(labels, [int(.15*len(labels)), int(.7*len(labels))])

#run model and test
model = rfc()
model = model.fit(dfTrn.values,DTrn)
model_score = model.score(dfDev,DDev)
print("Accuracy(%):",model_score*100)

#pretty accurate

Accuracy(%): 89.1373801917


In [197]:
import pandas as pd
import numpy
import pandas_montecarlo
from scipy.stats import shapiro, kruskal, f_oneway
from sklearn.ensemble import RandomForestClassifier as rfc
import random

## RandomForest Classifier with monte carlo simulated training set
random.seed(1000)

df = pd.read_csv("data.csv")
df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)

#split dataset for mc seed and testing

df, df_mc = numpy.split(df, [int(.7*len(df))])

#split dataset by class
df_1 = df_mc.loc[df_mc.diagnosis==1]
df_0 = df_mc.loc[df_mc.diagnosis==0]
df_1 = df_1.drop(["diagnosis"],axis=1)
df_0 = df_0.drop(["diagnosis"],axis=1)

#simulate class 0 data
mc_sim_df_0 = pd.DataFrame()
mc_sim_df_0['diagnosis']= ['0'] * len(df_0.index)
for col in df_0.columns:
    col_sim = df_0[col].montecarlo(sims = 2, bust = 0, goal = 0).data
    col_sim = col_sim.drop(["original"],axis = 1)
    for col2 in col_sim.columns:
        mc_sim_df_0[col]=col_sim[col2]

#simulate class 1 data
mc_sim_df_1 = pd.DataFrame()
mc_sim_df_1['diagnosis']= ['1'] * len(df_1.index)
for col in df_1.columns:
    print(col)
    col_sim = df_1[col].montecarlo(sims = 2, bust = 0, goal = 0).data
    col_sim = col_sim.drop(["original"],axis = 1)
    scores = []
    for col2 in col_sim.columns:
        mc_sim_df_1[col]=col_sim[col2]
        if(shapiro(mc_sim_df_1[col])[1]>0.05):
            print(kruskal(mc_sim_df_1[col],df_1[col]))
        else:
            print(f_oneway(mc_sim_df_1[col],df_1[col]))


#diag = mc_sim_df_1.append(mc_sim_df_0)['diagnosis']
mc_sim_df = mc_sim_df_1.append(mc_sim_df_0)
#shuffling dataframe for good luck
#mc_sim_df = mc_sim_df.sample(frac=1)
#mc_sim_df['diagnosis']=diag
mc_sim_df.head(20)


#values formatted
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)
dfDev, dfTes = numpy.split(df, [int(.7*len(df))])
DDev, DTes = numpy.split(labels, [int(.7*len(labels))])

DTrn =  mc_sim_df['diagnosis']
dfTrn = mc_sim_df.drop(['diagnosis'], axis = 1)

#run model and test
model = rfc()
model = model.fit(dfTrn.values,DTrn)

pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print(hit/len(pd))


radius_mean
KruskalResult(statistic=0.0, pvalue=1.0)
texture_mean
F_onewayResult(statistic=0.0, pvalue=1.0)
perimeter_mean
KruskalResult(statistic=0.0, pvalue=1.0)
area_mean
F_onewayResult(statistic=-3.252380956951368e-31, pvalue=nan)
smoothness_mean
KruskalResult(statistic=0.0, pvalue=1.0)
compactness_mean
KruskalResult(statistic=0.0, pvalue=1.0)
concavity_mean
KruskalResult(statistic=0.0, pvalue=1.0)
concave points_mean
KruskalResult(statistic=0.0, pvalue=1.0)
symmetry_mean
KruskalResult(statistic=0.0, pvalue=1.0)
fractal_dimension_mean
F_onewayResult(statistic=0.0, pvalue=1.0)
radius_se
F_onewayResult(statistic=-1.0283678012621201e-30, pvalue=nan)
texture_se
KruskalResult(statistic=0.0, pvalue=1.0)
perimeter_se
F_onewayResult(statistic=2.1196100275969638e-31, pvalue=1.0)
area_se
F_onewayResult(statistic=3.6085063032054577e-32, pvalue=1.0)
smoothness_se
KruskalResult(statistic=0.0, pvalue=1.0)
compactness_se
F_onewayResult(statistic=-4.781047788917236e-31, pvalue=nan)
concavity_se
F_

In [186]:
import pandas as pd
import numpy
import pandas_montecarlo
from scipy.stats import shapiro, kruskal, f_oneway
from sklearn.ensemble import RandomForestClassifier as rfc

## RandomForest Classifier with monte carlo simulated training set #edit: multi-set concatenation


df = pd.read_csv("data.csv")
df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)

#split dataset for mc seed and testing

df_mc, df = numpy.split(df, [int(.7*len(df))])

#split dataset by class
df_1 = df_mc.loc[df_mc.diagnosis==1]
df_0 = df_mc.loc[df_mc.diagnosis==0]
df_1 = df_1.drop(["diagnosis"],axis=1)
df_0 = df_0.drop(["diagnosis"],axis=1)

#simulate class 0 data
mc_sim_df_0 = pd.DataFrame()
mc_sim_df_0['diagnosis']= ['0'] * len(df_0.index)
for col in df_0.columns:
    col_sim = df_0[col].montecarlo(sims = 2, bust = 0, goal = 0).data
    col_sim = col_sim.drop(["original"],axis = 1)
    for col2 in col_sim.columns:
        mc_sim_df_0[col]=col_sim[col2]
        
##generate qq plot

#simulate class 1 data
mc_sim_df_1 = pd.DataFrame()
mc_sim_df_1['diagnosis']= ['1'] * len(df_1.index)
for col in df_1.columns:
    col_sim = df_1[col].montecarlo(sims = 2, bust = 0, goal = 0).data
    col_sim = col_sim.drop(["original"],axis = 1)
    scores = []
    for col2 in col_sim.columns:
        mc_sim_df_1[col]=col_sim[col2]
        kruskal_score = kruskal(df_1[col],mc_sim_df_1[col])
        scores.append((col_sim[col2],kruskal_score))
    


#diag = mc_sim_df_1.append(mc_sim_df_0)['diagnosis']
mc_sim_df = mc_sim_df_1.append(mc_sim_df_0)
#shuffling dataframe for good luck
#mc_sim_df = mc_sim_df.sample(frac=1)
#mc_sim_df['diagnosis']=diag
mc_sim_df.head(20)


#values formatted
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)
dfDev, dfTes = numpy.split(df, [int(.7*len(df))])
DDev, DTes = numpy.split(labels, [int(.7*len(labels))])

DTrn =  mc_sim_df['diagnosis']
dfTrn = mc_sim_df.drop(['diagnosis'], axis = 1)

#run model and test
model = rfc()
model = model.fit(dfTrn.values,DTrn)

pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print(hit/len(pd))


KeyError: 'radius_mean'

In [ ]:
#dataset normality

import pandas as pd
import numpy
from sklearn.svm import SVC
from scipy.stats import shapiro

#read in dataset
df = pd.read_csv("data.csv")
df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)
for col in df.columns:
    print(col.title(), shapiro(df[col]))
        
#dataset is normal


